In [19]:
import pandas as pd
import json, os
from collections import Counter

# =========================
# CONFIG
# =========================
INPUT_FILE = r"C:\Users\ahmed\Downloads\pharmacy_checkpoint.jsonl"
OUTPUT_CLEAN = "pharmacy_clean.csv"

TEXT_COL = "text"
INTENT_COL = "intent"
TOPIC_COL = "topic"
SENTIMENT_COL = "sentiment"

MIN_TEXT_LEN = 8

# =========================
# LOAD DATA
# =========================
if INPUT_FILE.endswith(".jsonl"):
    rows = []
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                rows.append(json.loads(line))
    df = pd.DataFrame(rows)
else:
    df = pd.read_csv(INPUT_FILE)

print(f"📥 Loaded rows: {len(df)}")

# =========================
# BASIC SANITY CHECK
# =========================
print("\n🧪 Missing values:")
print(df.isna().sum())

📥 Loaded rows: 2312

🧪 Missing values:
text         0
intent       0
topic        0
sentiment    0
dtype: int64


In [20]:


# =========================
# TEXT LENGTH CHECK
# =========================
df["text_len"] = df[TEXT_COL].str.len()
short_texts = df[df["text_len"] < MIN_TEXT_LEN]

print(f"\n✂️ Short texts (<{MIN_TEXT_LEN} chars): {len(short_texts)}")

# =========================
# DUPLICATE ANALYSIS
# =========================
total_rows = len(df)
unique_texts = df[TEXT_COL].nunique()

print("\n🔁 Duplicate analysis:")
print(f"Total rows   : {total_rows}")
print(f"Unique texts : {unique_texts}")
print(f"Duplicate %  : {(1 - unique_texts/total_rows) * 100:.2f}%")

dup_counts = (
    df.groupby(TEXT_COL)
      .size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)

repeated_texts = dup_counts[dup_counts["count"] > 1]

print(f"\n🔁 Repeated sentences: {len(repeated_texts)}")
display(repeated_texts.head(10))

# =========================
# LABEL CONFLICTS
# =========================
conflicts = (
    df.groupby(TEXT_COL)[INTENT_COL]
      .nunique()
      .reset_index()
      .query(f"{INTENT_COL} > 1")
)

print(f"\n⚠️ Intent conflicts: {len(conflicts)}")
display(conflicts.head(10))

# =========================
# DISTRIBUTIONS
# =========================
print("\n📊 Intent distribution:")
display(df[INTENT_COL].value_counts())

print("\n📊 Topic distribution:")
display(df[TOPIC_COL].value_counts())

print("\n📊 Sentiment distribution:")
display(df[SENTIMENT_COL].value_counts())

# =========================
# CLEANING
# =========================
clean_df = df.copy()

# Remove short texts
clean_df = clean_df[clean_df["text_len"] >= MIN_TEXT_LEN]

# Remove conflicting texts
clean_df = clean_df[~clean_df[TEXT_COL].isin(conflicts[TEXT_COL])]

# Remove duplicates (keep first)
clean_df = clean_df.drop_duplicates(subset=[TEXT_COL], keep="first")

clean_df_chunk_1 = clean_df.drop(columns=["text_len"])

print("\n🧹 After cleaning:")
print(f"Rows left: {len(clean_df)}")
print(f"Unique texts: {clean_df[TEXT_COL].nunique()}")

# =========================
# SAVE CLEAN DATA
# =========================
clean_df.to_csv(OUTPUT_CLEAN, index=False, encoding="utf-8-sig")
print(f"\n💾 Clean dataset saved to: {OUTPUT_CLEAN}")

# =========================
# FINAL QUALITY SCORE
# =========================
quality_score = (
    clean_df[TEXT_COL].nunique() / total_rows * 100
)

print(f"\n⭐ Dataset quality score: {quality_score:.2f}%")



✂️ Short texts (<8 chars): 0

🔁 Duplicate analysis:
Total rows   : 2312
Unique texts : 2284
Duplicate %  : 1.21%

🔁 Repeated sentences: 16


,text,count
1623,عندنا تأمين على الأدوية؟,13
1614,عندنا تأمين على الأدوية اللي بتشتريها من الصيد...,3
321,الانتظار طويل أوي، مفيش نظام في الصيدلية.,2
267,إيه نظام الدفع عند الاستلام؟,2
41,أسعار الأدوية غالية أوي، فين التخفيضات؟,2
1242,سعر (اسم الدواء) كام؟,2
1342,شكراً لكم!,2
1560,عندك فيتامينات لتقوية المناعة؟,2
1604,عندنا تأمين صحى على الأدوية؟,2
1607,عندنا تأمين صحي للادوية؟,2



⚠️ Intent conflicts: 0


,text,intent



📊 Intent distribution:


...


📊 Topic distribution:


topic
availability      569
delivery          514
staff_behavior    396
price             340
insurance         302
waiting_time      103
prescription       88
Name: count, dtype: int64


📊 Sentiment distribution:


sentiment
negative      828
neutral       794
positive      684
frustrated      3
mixed           2
angry           1
Name: count, dtype: int64


🧹 After cleaning:
Rows left: 2284
Unique texts: 2284

💾 Clean dataset saved to: pharmacy_clean.csv

⭐ Dataset quality score: 98.79%


In [21]:
INPUT_FILE = r"C:\Users\ahmed\Downloads\pharmacy_messages (1).csv"
OUTPUT_CLEAN = "pharmacy_clean_2.csv"

TEXT_COL = "text"
INTENT_COL = "intent"
TOPIC_COL = "topic"
SENTIMENT_COL = "sentiment"
MIN_TEXT_LEN = 8


df = pd.read_csv(INPUT_FILE)
print(f"📥 Loaded rows: {len(df)}")
print("\n🧪 Missing values:")
print(df.isna().sum())

📥 Loaded rows: 979

🧪 Missing values:
text         0
intent       0
topic        0
sentiment    0
dtype: int64


In [22]:
# TEXT LENGTH CHECK
# =========================
df["text_len"] = df[TEXT_COL].str.len()
short_texts = df[df["text_len"] < MIN_TEXT_LEN]

print(f"\n✂️ Short texts (<{MIN_TEXT_LEN} chars): {len(short_texts)}")

# =========================
# DUPLICATE ANALYSIS
# =========================
total_rows = len(df)
unique_texts = df[TEXT_COL].nunique()

print("\n🔁 Duplicate analysis:")
print(f"Total rows   : {total_rows}")
print(f"Unique texts : {unique_texts}")
print(f"Duplicate %  : {(1 - unique_texts/total_rows) * 100:.2f}%")

dup_counts = (
    df.groupby(TEXT_COL)
      .size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)

repeated_texts = dup_counts[dup_counts["count"] > 1]

print(f"\n🔁 Repeated sentences: {len(repeated_texts)}")
display(repeated_texts.head(10))

# =========================
# LABEL CONFLICTS
# =========================
conflicts = (
    df.groupby(TEXT_COL)[INTENT_COL]
      .nunique()
      .reset_index()
      .query(f"{INTENT_COL} > 1")
)

print(f"\n⚠️ Intent conflicts: {len(conflicts)}")
display(conflicts.head(10))

# =========================
# DISTRIBUTIONS
# =========================
print("\n📊 Intent distribution:")
display(df[INTENT_COL].value_counts())

print("\n📊 Topic distribution:")
display(df[TOPIC_COL].value_counts())

print("\n📊 Sentiment distribution:")
display(df[SENTIMENT_COL].value_counts())

# =========================
# CLEANING
# =========================
clean_df = df.copy()

# Remove short texts
clean_df = clean_df[clean_df["text_len"] >= MIN_TEXT_LEN]

# Remove conflicting texts
clean_df = clean_df[~clean_df[TEXT_COL].isin(conflicts[TEXT_COL])]

# Remove duplicates (keep first)
clean_df = clean_df.drop_duplicates(subset=[TEXT_COL], keep="first")

clean_df_chunk_2= clean_df.drop(columns=["text_len"])

print("\n🧹 After cleaning:")
print(f"Rows left: {len(clean_df)}")
print(f"Unique texts: {clean_df[TEXT_COL].nunique()}")

# =========================
# SAVE CLEAN DATA
# =========================
clean_df.to_csv(OUTPUT_CLEAN, index=False, encoding="utf-8-sig")
print(f"\n💾 Clean dataset saved to: {OUTPUT_CLEAN}")

# =========================
# FINAL QUALITY SCORE
# =========================
quality_score = (
    clean_df[TEXT_COL].nunique() / total_rows * 100
)

print(f"\n⭐ Dataset quality score: {quality_score:.2f}%")



✂️ Short texts (<8 chars): 0

🔁 Duplicate analysis:
Total rows   : 979
Unique texts : 979
Duplicate %  : 0.00%

🔁 Repeated sentences: 0


,text,count



⚠️ Intent conflicts: 0


,text,intent



📊 Intent distribution:


...


📊 Topic distribution:


topic
delivery          434
staff_behavior    309
insurance         149
waiting_time       87
Name: count, dtype: int64


📊 Sentiment distribution:


sentiment
negative    338
positive    326
neutral     315
Name: count, dtype: int64


🧹 After cleaning:
Rows left: 979
Unique texts: 979

💾 Clean dataset saved to: pharmacy_clean_2.csv

⭐ Dataset quality score: 100.00%


In [23]:
combined_df = pd.concat([clean_df_chunk_1, clean_df_chunk_2], ignore_index=True)

print(f"📊 Combined dataset:")
print(f"Total rows: {len(combined_df)}")
print(f"Columns: {combined_df.columns.tolist()}")
print(f"\n{combined_df.head()}")

# Optional: Remove duplicates from combined dataset
combined_df = combined_df.drop_duplicates(subset=[TEXT_COL], keep="first")

print(f"\n✅ After removing duplicates from combined dataset:")
print(f"Total rows: {len(combined_df)}")

# Save combined dataset
combined_df.to_csv("pharmacy_combined.csv", index=False, encoding="utf-8-sig")
print(f"\n💾 Combined dataset saved to: pharmacy_combined.csv")

📊 Combined dataset:
Total rows: 3263
Columns: ['text', 'intent', 'topic', 'sentiment']

                                                text   intent         topic  \
0  الصيدلية دي أحسن صيدلية شفتها! كل المنتجات موج...   Praise  availability   
1  أنا أحب الصيدلية دي كتير! كل مرة بجيلها لاقى ك...   Praise  availability   
2  الصيدلية دي جوه عيني! دايماً متوفرة على كل الل...   Praise  availability   
3                 مرحبا! هل لديكم دواء (اسم الدواء)؟  Inquiry  availability   
4                هل يمكنني شراء هذا الدواء بسعر أقل؟  Inquiry         price   

  sentiment  
0  positive  
1  positive  
2  positive  
3   neutral  
4   neutral  

✅ After removing duplicates from combined dataset:
Total rows: 2284

💾 Combined dataset saved to: pharmacy_combined.csv


In [24]:
df = combined_df.copy()


In [25]:
# TEXT LENGTH CHECK
# =========================
df["text_len"] = df[TEXT_COL].str.len()
short_texts = df[df["text_len"] < MIN_TEXT_LEN]

print(f"\n✂️ Short texts (<{MIN_TEXT_LEN} chars): {len(short_texts)}")

# =========================
# DUPLICATE ANALYSIS
# =========================
total_rows = len(df)
unique_texts = df[TEXT_COL].nunique()

print("\n🔁 Duplicate analysis:")
print(f"Total rows   : {total_rows}")
print(f"Unique texts : {unique_texts}")
print(f"Duplicate %  : {(1 - unique_texts/total_rows) * 100:.2f}%")

dup_counts = (
    df.groupby(TEXT_COL)
      .size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)

repeated_texts = dup_counts[dup_counts["count"] > 1]

print(f"\n🔁 Repeated sentences: {len(repeated_texts)}")
display(repeated_texts.head(10))

# =========================
# LABEL CONFLICTS
# =========================
conflicts = (
    df.groupby(TEXT_COL)[INTENT_COL]
      .nunique()
      .reset_index()
      .query(f"{INTENT_COL} > 1")
)

print(f"\n⚠️ Intent conflicts: {len(conflicts)}")
display(conflicts.head(10))

# =========================
# DISTRIBUTIONS
# =========================
print("\n📊 Intent distribution:")
display(df[INTENT_COL].value_counts())

print("\n📊 Topic distribution:")
display(df[TOPIC_COL].value_counts())

print("\n📊 Sentiment distribution:")
display(df[SENTIMENT_COL].value_counts())

# =========================
# CLEANING
# =========================
clean_df = df.copy()

# Remove short texts
clean_df = clean_df[clean_df["text_len"] >= MIN_TEXT_LEN]

# Remove conflicting texts
clean_df = clean_df[~clean_df[TEXT_COL].isin(conflicts[TEXT_COL])]

# Remove duplicates (keep first)
clean_df = clean_df.drop_duplicates(subset=[TEXT_COL], keep="first")

clean_df_chunk_2= clean_df.drop(columns=["text_len"])

print("\n🧹 After cleaning:")
print(f"Rows left: {len(clean_df)}")
print(f"Unique texts: {clean_df[TEXT_COL].nunique()}")

# =========================
# SAVE CLEAN DATA
# =========================
clean_df.to_csv(OUTPUT_CLEAN, index=False, encoding="utf-8-sig")
print(f"\n💾 Clean dataset saved to: {OUTPUT_CLEAN}")

# =========================
# FINAL QUALITY SCORE
# =========================
quality_score = (
    clean_df[TEXT_COL].nunique() / total_rows * 100
)

print(f"\n⭐ Dataset quality score: {quality_score:.2f}%")



✂️ Short texts (<8 chars): 0

🔁 Duplicate analysis:
Total rows   : 2284
Unique texts : 2284
Duplicate %  : 0.00%

🔁 Repeated sentences: 0


,text,count



⚠️ Intent conflicts: 0


,text,intent



📊 Intent distribution:


...


📊 Topic distribution:


topic
availability      567
delivery          511
staff_behavior    396
price             337
insurance         284
waiting_time      102
prescription       87
Name: count, dtype: int64


📊 Sentiment distribution:


sentiment
negative      826
neutral       769
positive      683
frustrated      3
mixed           2
angry           1
Name: count, dtype: int64


🧹 After cleaning:
Rows left: 2284
Unique texts: 2284

💾 Clean dataset saved to: pharmacy_clean_2.csv

⭐ Dataset quality score: 100.00%


In [10]:
# ===============================
import os, yaml
import pandas as pd

EXPORT_DIR = "saved_models"
os.makedirs(EXPORT_DIR, exist_ok=True)

# 1) Load
df = pd.read_csv("pharmacy_combined.csv").dropna(subset=["text","intent","topic","sentiment"]).copy()

print("Before:")
print("topic:\n", df["topic"].value_counts(), "\n")
print("sentiment:\n", df["sentiment"].value_counts(), "\n")
print("intent:\n", df["intent"].value_counts(), "\n")

# 2) Sentiment -> 3-class (match your project)
sent_map = {"angry": "negative", "frustrated": "negative", "mixed": "neutral"}
df["sentiment"] = df["sentiment"].replace(sent_map)

# 3) Topic merge (your requested fix)
topic_merge_map = {
    "waiting_time": "delivery",
    "prescription": "insurance",
}
df["topic"] = df["topic"].replace(topic_merge_map)

print("After merge:")
print("topic:\n", df["topic"].value_counts(), "\n")

# 4) (Optional) filter to allowed labels only (keeps things clean)
allowed_topics = ["availability","delivery","staff_behavior","price","insurance"]  # now 5 topics
allowed_sentiments = ["negative","neutral","positive"]
allowed_intents = ["Inquiry","Complaint","Praise"]  # keep your dataset intent set

before = len(df)
df = df[
    df["topic"].isin(allowed_topics) &
    df["sentiment"].isin(allowed_sentiments) &
    df["intent"].isin(allowed_intents)
].copy()
print(f"Dropped {before-len(df)} rows outside allowed label sets.\n")

# 5) Save fixed dataset
out_csv = "pharmacy_combined_fixed_merge.csv"
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"✅ Saved: {out_csv}")

Before:
topic:
 topic
availability      567
delivery          511
staff_behavior    396
price             337
insurance         284
waiting_time      102
prescription       87
Name: count, dtype: int64 

sentiment:
 sentiment
negative      826
neutral       769
positive      683
frustrated      3
mixed           2
angry           1
Name: count, dtype: int64 

intent:
 intent
Complaint    842
Inquiry      766
Praise       676
Name: count, dtype: int64 

After merge:
topic:
 topic
delivery          613
availability      567
staff_behavior    396
insurance         371
price             337
Name: count, dtype: int64 

Dropped 0 rows outside allowed label sets.

✅ Saved: pharmacy_combined_fixed_merge.csv


In [11]:
print(df["intent"].value_counts())
print(df["topic"].value_counts())
print(df["sentiment"].value_counts())

intent
Complaint    842
Inquiry      766
Praise       676
Name: count, dtype: int64
topic
delivery          613
availability      567
staff_behavior    396
insurance         371
price             337
Name: count, dtype: int64
sentiment
negative    830
neutral     771
positive    683
Name: count, dtype: int64


In [9]:
import pandas as pd

df = pd.read_csv("pharmacy_combined.csv").dropna(subset=["text","intent","topic","sentiment"]).copy()

# sentiment -> 3-class
df["sentiment"] = df["sentiment"].replace({
    "angry": "negative",
    "frustrated": "negative",
    "mixed": "neutral",
})

# topic merge (your rules)
df["topic"] = df["topic"].replace({
    "waiting_time": "delivery",
    "prescription": "insurance",
})

# keep Praise (don't drop it)
allowed_intents = ["Inquiry", "Complaint", "Praise"]
allowed_sent = ["negative","neutral","positive"]
allowed_topics = ["availability","delivery","staff_behavior","price","insurance"]

before = len(df)
df = df[
    df["intent"].isin(allowed_intents) &
    df["sentiment"].isin(allowed_sent) &
    df["topic"].isin(allowed_topics)
].copy()

print("Dropped rows:", before - len(df))  # should be 0 (or very small)
df.to_csv("pharmacy_combined_fixed.csv", index=False, encoding="utf-8-sig")
print("✅ Saved: pharmacy_combined_fixed.csv")
print(df["intent"].value_counts())
print(df["topic"].value_counts())
print(df["sentiment"].value_counts())


Dropped rows: 0
✅ Saved: pharmacy_combined_fixed.csv
intent
Complaint    842
Inquiry      766
Praise       676
Name: count, dtype: int64
topic
delivery          613
availability      567
staff_behavior    396
insurance         371
price             337
Name: count, dtype: int64
sentiment
negative    830
neutral     771
positive    683
Name: count, dtype: int64
